## Working with annotation volumes

Annotation volumes are useful references for understanding brain parcellations at multiple levels. For example, a cell's location can be described as being in the lateral geniculate nucleus or in any of its parent structures in the anatomical ontology (dorsal thalamus, thalamus, diencephalon, etc.). Annotation volumes can label CCF-registered data with terms from the [HOMBA ontology](../docs/HOMBA_ontology_v1.md). 

This tutorial shows some examples for using the annotation volumes to (1) query labels and (2) extract regions at any level as a mask for reference and visualization. 